In [0]:
%pip install -r ../requirements.txt
dbutils.library.restartPython()

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
from autogluon.timeseries import TimeSeriesDataFrame, TimeSeriesPredictor
from sklearn.metrics._regression import mean_absolute_error
import logging
logging.getLogger("py4j").setLevel(logging.ERROR)

In [0]:
# Set Dependencies 
CATALOG =  "main"
SCHEMA = "sgfs"
TABLE = "grocery_data"

Download the date we are going to use for the exercise it is available at the below link [https://autogluon.s3.amazonaws.com/datasets/timeseries/grocery_sales/test.csv](below link )

Below cell contains code snippet to recreate a new delta table if required

In [0]:
# # Define the CSV URL
# csv_url = "https://autogluon.s3.amazonaws.com/datasets/timeseries/grocery_sales/test.csv"

# ! wget {csv_url} -O /local_disk0/test.csv

# # Read the CSV file into a Spark DataFrame.
# # The options header=True and inferSchema=True allow Spark to use the first row as header and automatically determine the data types.
# df = spark.read.format("csv") \
#     .option("header", "true") \
#     .option("inferSchema", "true") \
#     .load("file:///local_disk0/test.csv")

# df.write.mode('overwrite').saveAsTable(f"{CATALOG}.{SCHEMA}.{TABLE}")

In [0]:

df = spark.sql(f"SELECT * FROM {CATALOG}.{SCHEMA}.{TABLE}").toPandas()
df.head()

,item_id,timestamp,scaled_price,promotion_email,promotion_homepage,unit_sales
0,1062_101,2018-01-01,0.879130,0.0,0.0,636.0
1,1062_101,2018-01-08,0.994517,0.0,0.0,123.0
2,1062_101,2018-01-15,1.005513,0.0,0.0,391.0
3,1062_101,2018-01-22,1.000000,0.0,0.0,339.0
4,1062_101,2018-01-29,0.883309,0.0,0.0,661.0


In [0]:
data = TimeSeriesDataFrame.from_data_frame(df, timestamp_column='timestamp', id_column='item_id')
data.head()

scaled_price  ...  unit_sales
item_id  timestamp                 ...            
1062_101 2018-01-01      0.879130  ...       636.0
         2018-01-08      0.994517  ...       123.0
         2018-01-15      1.005513  ...       391.0
         2018-01-22      1.000000  ...       339.0
         2018-01-29      0.883309  ...       661.0

[5 rows x 4 columns]

In [0]:
from sklearn.metrics import mean_absolute_error
prediction_length = 8
train_data, test_data = data.train_test_split(prediction_length)

predictor = TimeSeriesPredictor(prediction_length=prediction_length,
                                target='unit_sales',
                                eval_metric='RMSE',
                                path = '/local_disk0/autogluon'
                                # known_covariates_names=["scaled_price", "promotion_email", "promotion_homepage"]
                                ).fit(
    train_data,
    hyperparameters={
        "DeepAR": {},
        "AutoARIMA": {},
        "PatchTST": {},
        "Chronos": [
            # Zero-shot model WITHOUT covariates
            {
                "model_path": "bolt_small",
                "ag_args": {"name_suffix": "ZeroShot"},
            },
            {
            # fine tune the base model without the covariates
                "model_path": "bolt_small", 
                "fine_tune": True, 
                "ag_args": {"name_suffix": "FineTuned"},
            }
        ],
    },
    enable_ensemble=False,
    time_limit=300,
)

Beginning AutoGluon training... Time limit = 300s
AutoGluon will save models to '/local_disk0/autogluon'
=================== System Info ===================
AutoGluon Version:  1.2
Python Version:     3.12.3
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #84~20.04.1-Ubuntu SMP Mon Nov 4 18:58:41 UTC 2024
CPU Count:          8
GPU Count:          0
Memory Avail:       13.25 GB / 22.47 GB (59.0%)
Disk Space Avail:   160.32 GB / 180.05 GB (89.0%)

Fitting with arguments:
{'enable_ensemble': False,
 'eval_metric': RMSE,
 'hyperparameters': {'AutoARIMA': {},
                     'Chronos': [{'ag_args': {'name_suffix': 'ZeroShot'},
                                  'model_path': 'bolt_small'},
                                 {'ag_args': {'name_suffix': 'FineTuned'},
                                  'fine_tune': True,
                                  'model_path': 'bolt_small'}],
                     'DeepAR': {},
                     'PatchTST': {}},
 'known_cova

[2025-02-13 12:16:30,919] [WARNING] [real_accelerator.py:181:get_accelerator] Setting accelerator to CPU. If you have GPU or other accelerator, we were unable to detect it.
[2025-02-13 12:16:30,926] [INFO] [real_accelerator.py:222:get_accelerator] Setting ds_accelerator to cpu (auto detect)


/local_disk0/.ephemeral_nfs/envs/pythonEnv-1df25791-eeb3-4a87-9518-bab9a98a03c5/lib/python3.12/site-packages/accelerate/state.py:245: UserWarning: OMP_NUM_THREADS/MKL_NUM_THREADS unset, we set it at 8 to improve oob performance.
  warnings.warn(
/usr/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
[rank0]:[W213 12:16:35.706594578 reducer.cpp:1400] Warning: find_unused_parameters=True was specified in DDP constructor, but did not find any unused parameters in the forward pass. This flag results in an extra traversal of the autograd graph every iteration,  which can adversely affect performance. If your model indeed never has any unused parameters in the forward pass, consider turning this flag off. Note that this warning may be a false positive if your model has flow control causing later iterations to have unused parameters. (function operator())
	Saving fine-tuned model to /local_disk0/autogluon/models/ChronosFineTuned[bolt_small]/W0/fin

In [0]:
predictor.leaderboard(test_data)

Additional data provided, testing on additional data. Resulting leaderboard will be sorted according to test score (`score_test`).
INFO:root:Exception while sending command.
Traceback (most recent call last):
  File "/databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 528, in send_command
    self.socket.sendall(command.encode("utf-8"))
ConnectionResetError: [Errno 104] Connection reset by peer

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 531, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while sending


,model,score_test,score_val,pred_time_test,pred_time_val,fit_time_marginal,fit_order
0,ChronosFineTuned[bolt_small],-223.036973,-398.807604,0.830151,0.487855,73.247818,3
1,ChronosZeroShot[bolt_small],-225.090169,-418.831047,1.100749,1.079132,8.047948,2
2,PatchTST,-244.674664,-401.624772,0.288992,0.194946,45.086600,5
3,DeepAR,-254.694776,-394.551332,3.784090,1.308038,53.539626,4
4,AutoARIMA,-705.911009,-409.216136,17.263526,24.608125,0.056134,1


In [0]:
result_set = spark.createDataFrame(predictor.predict(train_data, model='ChronosFineTuned[bolt_small]').to_data_frame().reset_index())
display(result_set)

item_id,timestamp,mean,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9
1062_101,2018-06-11T00:00:00Z,318.23038,207.78197,234.272,260.10175,288.8349,318.23038,353.3537,393.51236,446.2955,529.04346
1062_101,2018-06-18T00:00:00Z,320.65714,204.83533,227.83041,253.5557,285.0191,320.65714,360.4784,406.12372,464.7505,553.5131
1062_101,2018-06-25T00:00:00Z,322.06042,201.93448,223.72017,250.26973,282.83945,322.06042,364.69003,410.62186,469.8174,556.3086
1062_101,2018-07-02T00:00:00Z,327.35907,204.56335,227.43904,254.08304,287.07172,327.35907,370.492,419.20035,479.36047,567.4683
1062_101,2018-07-09T00:00:00Z,335.30338,207.72115,231.1143,257.4565,292.40686,335.30338,383.39764,435.09735,495.24786,583.6441
1062_101,2018-07-16T00:00:00Z,337.76657,208.27238,229.1824,255.87311,291.78888,337.76657,386.46002,440.25223,502.59854,596.0515
1062_101,2018-07-23T00:00:00Z,335.94363,206.64932,229.9085,256.01425,290.03644,335.94363,385.7055,442.4121,506.05402,598.949
1062_101,2018-07-30T00:00:00Z,335.10605,206.20715,231.03065,256.34167,290.31363,335.10605,386.9864,442.94794,508.27774,601.1154
1062_102,2018-06-11T00:00:00Z,272.0662,166.90251,200.31992,226.26439,250.29321,272.0662,296.5829,323.50397,357.14264,410.60855
1062_102,2018-06-18T00:00:00Z,272.08264,156.69818,191.93951,220.31203,246.78424,272.08264,298.53232,328.95416,366.87402,427.32767


In [0]:
result_set.write.mode("overwrite").saveAsTable(f"{CATALOG}.{SCHEMA}.grocery_data_forecast") 